<a href="https://colab.research.google.com/github/fatemafaria142/Natural-Language-Understanding-in-English-with-MultiNLI-Corpus/blob/main/Natural_Language_Inference_using_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00


# **Load the required packages**

In [2]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

### **Dataset Link:** https://huggingface.co/datasets/multi_nli

In [3]:
dataset="multi_nli"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-NLI-v1"

# **Dataset preparation**

In [10]:
def prepare_train_data(data_id, num_samples=5000):
    data = load_dataset(data_id, split="train")

    # Take the first 5000 rows
    data = data.select(range(min(num_samples, len(data))))
    instructions_template = "Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION."

    # Extract the required columns
    data_df = data.to_pandas()
    data_df = data_df[["premise", "hypothesis", "label"]]
    data_df["text"] = data_df[["premise","hypothesis","label"]].apply(lambda x: "user\n" + instructions_template + "  \n Premise\n" + x["premise"] +  "\n Hypothesis\n" + x["hypothesis"]+ "\n Label\n" + str(x["label"]), axis=1)
    data = Dataset.from_pandas(data_df)

    return data



In [11]:
data = prepare_train_data(dataset, num_samples=5000)

In [12]:
data

Dataset({
    features: ['premise', 'hypothesis', 'label', 'text'],
    num_rows: 5000
})

In [13]:
data[0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'text': 'user\nEvaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.  \n Premise\nConceptually cream skimming has two basic dimensions - product and geography.\n Hypothesis\nProduct and geography are what make cream skimming work. \n Label\n1'}

In [15]:
data[1]

{'premise': 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him',
 'hypothesis': 'You lose the things to the following level if the people recall.',
 'label': 0,
 'text': 'user\nEvaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.  \n Premise\nyou know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the pa

In [16]:
data[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0,
 'text': 'user\nEvaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.  \n Premise\nOne of our number will carry out your instructions minutely.\n Hypothesis\nA member of my team will execute your orders with immense precision.\n Label\n0'}

## **We have to model the Model (not the base version)**

In [17]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [18]:
model, tokenizer = get_model_and_tokenizer(model_id)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# **Setting up the LoRA**

In [19]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [20]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [21]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [22]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.391200
20,1.820600
30,1.207500
40,0.880400
50,0.773400
60,0.808300
70,0.654500
80,0.725600
90,0.722400
100,0.789900


TrainOutput(global_step=250, training_loss=0.8655345039367676, metrics={'train_runtime': 197.804, 'train_samples_per_second': 5.056, 'train_steps_per_second': 1.264, 'total_flos': 1031545939525632.0, 'train_loss': 0.8655345039367676, 'epoch': 0.2})

# **Merging the LoRA with the base model**

In [23]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/tinyllama-NLI-v1/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [24]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

# **Inference from the LLM for some examples**

In [25]:
def formatted_prompt(instruction, premise, hypothesis)-> str:
    return f"<|im_start|>Instructions:\n{instruction}<|im_end|>\n<|im_start|>Premise:\n{premise:}<|im_end|>\n<|im_start|>Hypothesis:\n{hypothesis}<|im_end|>\n<|im_start|>Label:"

In [26]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(instruction, premise_input,  hypothesis_input):

    prompt = formatted_prompt(instruction, premise_input, hypothesis_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=1024,
        pad_token_id=tokenizer.eos_token_id
    )
    start_time = perf_counter()

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()  # Remove leading and trailing whitespaces
    print(decoded_output)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")

#### **Example No: 1**

In [28]:
instructions_template = "Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION."

Premise = "These organizations invest the time and effort to understand their processes and how those processes contribute to or hamper mission accomplishment."
Hypothesis = "These organizations invest lots of time to understand how some processes can contribute to or haampe"

# 0 entailment -> Ground Truth
generate_response(instruction=instructions_template, premise_input=Premise , hypothesis_input=Hypothesis)


<|im_start|>Instructions:
Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.<|im_end|>
<|im_start|>Premise:
These organizations invest the time and effort to understand their processes and how those processes contribute to or hamper mission accomplishment.<|im_end|>
<|im_start|>Hypothesis:
These organizations invest lots of time to understand how some processes can contribute to or haampe<|im_end|>
<|im_start|>Label:
0
Time taken for inference: 0.12 seconds


#### **Example No: 2**

In [30]:
instructions_template = "Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION."

Premise = "i bet even my cats could do that"
Hypothesis = "My cats could probably do that because they are brilliant."
# 1 neutral -> Ground Truth
generate_response(instruction=instructions_template, premise_input=Premise , hypothesis_input=Hypothesis)


<|im_start|>Instructions:
Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.<|im_end|>
<|im_start|>Premise:
i bet even my cats could do that<|im_end|>
<|im_start|>Hypothesis:
My cats could probably do that because they are brilliant.<|im_end|>
<|im_start|>Label:
2
Time taken for inference: 0.14 seconds


#### **Example No: 3**

In [29]:
instructions_template = "Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION."

Premise = "One of our number will carry out your instructions minutely."
Hypothesis = "A member of my team will execute your orders with immense precision."
# 0 entailment -> Ground Truth
generate_response(instruction=instructions_template, premise_input=Premise , hypothesis_input=Hypothesis)


<|im_start|>Instructions:
Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.<|im_end|>
<|im_start|>Premise:
One of our number will carry out your instructions minutely.<|im_end|>
<|im_start|>Hypothesis:
A member of my team will execute your orders with immense precision.<|im_end|>
<|im_start|>Label:
0
Time taken for inference: 0.23 seconds


#### **Example No: 4**

In [31]:
instructions_template = "Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION."

Premise = "we have gone on trips we've bathed in streams"
Hypothesis = "In addition to bathing in streams, we've also gone to spas and saunas."

# 1 neutral -> Ground Truth
generate_response(instruction=instructions_template, premise_input=Premise , hypothesis_input=Hypothesis)


<|im_start|>Instructions:
Evaluate the relationship between the given premise and hypothesis. Determine if the hypothesis logically follows from the premise, contradicts it, or is unrelated. The premise provides the information for the model to base its reasoning, and the hypothesis is the statement to be evaluated based on the provided premise. Assign a label to the relationship between the premise and hypothesis: 1 for NEUTRAL, 0 for ENTAILMENT, and 2 for CONTRADICTION.<|im_end|>
<|im_start|>Premise:
we have gone on trips we've bathed in streams<|im_end|>
<|im_start|>Hypothesis:
In addition to bathing in streams, we've also gone to spas and saunas.<|im_end|>
<|im_start|>Label:
0
Time taken for inference: 0.2 seconds
